![Character Embedding](https://cdn-images-1.medium.com/max/1000/0*AgFPi9E5iqGju--j.jpg)

Photo Credit: https://cdn.pixabay.com/photo/2016/11/30/12/16/question-mark-1872665_960_720.jpg

# Character Embedding
In 2013, Tomas Mikolov introduced word embedding to learn a better quality of word. At that time word2vec is state of the art on dealing with text. Later on, doc2vec is introduced as well. What if we think in another angel? Instead of aggregate from word to document, is it possible to aggregate from character to word. 

In this article, you will go through what, why, when and how on Character Embedding

**What?**

Text Understanding from Scratch introduced using character CNN. In Char CNN paper, a list of character are defined 70 characters which including 26 English letters, 10 digits, 33 special characters and new line character.

On the other hand, Google Brain team introduced Exploring the Limits of Language Modeling and released the lm_1b model which includes 256 vectors (including 52 characters, special characters) and the dimension is just 16. By comparing to word embedding, the dimension can increase up to 300 while the number of vectors is huge.

**Why?**

In english, all words are formed by 26 (or 52 if including both upper and lower case character, or even more if including special characters). Having the character embedding, every single word's vector can be formed even it is out-of-vocabulary words (optional). On the other than, word2vec embedding can only handle those seen words.

Another benefit is that it good fits for misspelling words, emoticons, new words (e.g. In 2018, Oxford English Dictionary introduced new word which is boba tea 波霸奶茶. Before that we do not have any pre-trained word embedding for that).
It handles infrequent words better than word2vec embedding as later one suffers from lack of enough training opportunity for those rare words.

Third reason is that as there are only small amount of vector, it reduces model complexity and improving the performance (in terms of speed)

**When?**

In NLP, we can apply character embedding on:
1. [Text Classification](https://arxiv.org/pdf/1509.01626.pdf)
2. [Language Model](https://arxiv.org/pdf/1602.02410.pdf)
3. [Named Entity Recognition](https://www.aclweb.org/anthology/Q16-1026)

In [1]:
import re
import pandas as pd
import numpy as np
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Preprocessing
import nltk
from nltk.tokenize import sent_tokenize

# Modeling
import keras
from keras.models import Model, load_model
from keras.layers import Dense, Input, Dropout, MaxPooling1D, Conv1D, GlobalMaxPool1D, Bidirectional
from keras.layers import LSTM, Lambda, Bidirectional, concatenate, BatchNormalization, Embedding
from keras.layers import TimeDistributed
from keras.optimizers import Adam
import tensorflow as tf
import keras.backend as K

import IPython
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Data Ingestion

In [2]:
def get_raw_data():
    # Get file from http://archive.ics.uci.edu/ml/datasets/News+Aggregator
    df = pd.read_csv(
        "./../data/demo_data/NewsAggregatorDataset/newsCorpora.csv", 
        sep='\t', 
        names=['id', 'headline', 'url', 'publisher', 'category', 'story', 'hostname', 'timestamp']
    )
    
    # Category: b = business, t = science and technology, e = entertainment, m = health
    return df[['category', 'headline']]

In [3]:
df = get_raw_data()
df.head()

,category,headline
0,b,"Fed official says weak data caused by weather,..."
1,b,Fed's Charles Plosser sees high bar for change...
2,b,US open: Stocks fall after Fed official hints ...
3,b,"Fed risks falling 'behind the curve', Charles ..."
4,b,Fed's Plosser: Nasty Weather Has Curbed Job Gr...


In [4]:
print('Number of news: %d' % (len(df)))

Number of news: 422419


In [5]:
train_df, test_df = train_test_split(df, test_size=0.2)

# Modeling

**_Character Embedding_**
1. Define a list a characters (i.e. m). For example, can use alphanumeric and some special characters. For my example, English characters (52), number (10), special characters (20) and one unknown character, UNK. Total 83 characters.
2. Transfer characters as 1-hot encoding and got a sequence for vectors. For unknown characters and blank characters, use all-zero vector to replace it. If exceeding pre-defined maximum length of characters (i.e. l), ignoring it. The output is 16 dimensions vector per every single character.
3. Using 3 1D CNN layers (configurable) to learn the sequence

**_Sentence Embedding_**
1. Bi-directional LSTM followed CNN layers
2. Some dropout layers are added after LSTM

In [ ]:
class CharCNN:
    __author__ = "Edward Ma"
    __copyright__ = "Copyright 2018, Edward Ma"
    __credits__ = ["Edward Ma"]
    __license__ = "Apache"
    __version__ = "2.0"
    __maintainer__ = "Edward Ma"
    __email__ = "makcedward@gmail.com"
    
    CHAR_DICT = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .!?:,\'%-\(\)/$|&;[]"'
    
    def __init__(self, max_len_of_sentence, max_num_of_setnence, verbose=10):
        self.max_len_of_sentence = max_len_of_sentence
        self.max_num_of_setnence = max_num_of_setnence
        self.verbose = verbose
        
        self.num_of_char = 0
        self.num_of_label = 0
        self.unknown_label = ''
        
    def build_char_dictionary(self, char_dict=None, unknown_label='UNK'):
        """
            Define possbile char set. Using "UNK" if character does not exist in this set
        """ 
        
        if char_dict is None:
            char_dict = self.CHAR_DICT
            
        self.unknown_label = unknown_label

        chars = []

        for c in char_dict:
            chars.append(c)

        chars = list(set(chars))
        
        chars.insert(0, unknown_label)

        self.num_of_char = len(chars)
        self.char_indices = dict((c, i) for i, c in enumerate(chars))
        self.indices_char = dict((i, c) for i, c in enumerate(chars))
        
        if self.verbose > 5:
            print('Totoal number of chars:', self.num_of_char)

            print('First 3 char_indices sample:', {k: self.char_indices[k] for k in list(self.char_indices)[:3]})
            print('First 3 indices_char sample:', {k: self.indices_char[k] for k in list(self.indices_char)[:3]})
            

        return self.char_indices, self.indices_char, self.num_of_char
    
    def convert_labels(self, labels):
        """
            Convert label to numeric
        """
        self.label2indexes = dict((l, i) for i, l in enumerate(labels))
        self.index2labels = dict((i, l) for i, l in enumerate(labels))

        if self.verbose > 5:
            print('Label to Index: ', self.label2indexes)
            print('Index to Label: ', self.index2labels)
            
        self.num_of_label = len(self.label2indexes)

        return self.label2indexes, self.index2labels
    
    def _transform_raw_data(self, df, x_col, y_col, label2indexes=None, sample_size=None):
        """
            ##### Transform raw data to list
        """
        
        x = []
        y = []

        actual_max_sentence = 0
        
        if sample_size is None:
            sample_size = len(df)

        for i, row in df.head(sample_size).iterrows():
            x_data = row[x_col]
            y_data = row[y_col]

            sentences = sent_tokenize(x_data)
            x.append(sentences)

            if len(sentences) > actual_max_sentence:
                actual_max_sentence = len(sentences)

            y.append(label2indexes[y_data])

        if self.verbose > 5:
            print('Number of news: %d' % (len(x)))
            print('Actual max sentence: %d' % actual_max_sentence)

        return x, y
    
    def _transform_training_data(self, x_raw, y_raw, max_len_of_sentence=None, max_num_of_setnence=None):
        """
            ##### Transform preorcessed data to numpy
        """
        unknown_value = self.char_indices[self.unknown_label]
        
        x = np.ones((len(x_raw), max_num_of_setnence, max_len_of_sentence), dtype=np.int64) * unknown_value
        y = np.array(y_raw)
        
        if max_len_of_sentence is None:
            max_len_of_sentence = self.max_len_of_sentence
        if max_num_of_setnence is None:
            max_num_of_setnence = self.max_num_of_setnence

        for i, doc in enumerate(x_raw):
            for j, sentence in enumerate(doc):
                if j < max_num_of_setnence:
                    for t, char in enumerate(sentence[-max_len_of_sentence:]):
                        if char not in self.char_indices:
                            x[i, j, (max_len_of_sentence-1-t)] = self.char_indices['UNK']
                        else:
                            x[i, j, (max_len_of_sentence-1-t)] = self.char_indices[char]

        return x, y

    def _build_character_block(self, block, dropout=0.3, filters=[64, 100], kernel_size=[3, 3], 
                         pool_size=[2, 2], padding='valid', activation='relu', 
                         kernel_initializer='glorot_normal'):
        
        for i in range(len(filters)):
            block = Conv1D(
                filters=filters[i], kernel_size=kernel_size[i],
                padding=padding, activation=activation, kernel_initializer=kernel_initializer)(block)

        block = Dropout(dropout)(block)
        block = MaxPooling1D(pool_size=pool_size[i])(block)

        block = GlobalMaxPool1D()(block)
        block = Dense(128, activation='relu')(block)
        return block
    
    def _build_sentence_block(self, max_len_of_sentence, max_num_of_setnence, 
                              char_dimension=16,
                              filters=[[3, 5, 7], [200, 300, 300], [300, 400, 400]], 
#                               filters=[[100, 200, 200], [200, 300, 300], [300, 400, 400]], 
                              kernel_sizes=[[4, 3, 3], [5, 3, 3], [6, 3, 3]], 
                              pool_sizes=[[2, 2, 2], [2, 2, 2], [2, 2, 2]],
                              dropout=0.4):
        
        sent_input = Input(shape=(max_len_of_sentence, ), dtype='int64')
        embedded = Embedding(self.num_of_char, char_dimension, input_length=max_len_of_sentence)(sent_input)
        
        blocks = []
        for i, filter_layers in enumerate(filters):
            blocks.append(
                self._build_character_block(
                    block=embedded, filters=filters[i], kernel_size=kernel_sizes[i], pool_size=pool_sizes[i])
            )

        sent_output = concatenate(blocks, axis=-1)
        sent_output = Dropout(dropout)(sent_output)
        sent_encoder = Model(inputs=sent_input, outputs=sent_output)

        return sent_encoder
    
    def _build_document_block(self, sent_encoder, max_len_of_sentence, max_num_of_setnence, 
                             num_of_label, dropout=0.3, 
                             loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']):
        doc_input = Input(shape=(max_num_of_setnence, max_len_of_sentence), dtype='int64')
        doc_output = TimeDistributed(sent_encoder)(doc_input)

        doc_output = Bidirectional(LSTM(128, return_sequences=False, dropout=dropout))(doc_output)

        doc_output = Dropout(dropout)(doc_output)
        doc_output = Dense(128, activation='relu')(doc_output)
        doc_output = Dropout(dropout)(doc_output)
        doc_output = Dense(num_of_label, activation='sigmoid')(doc_output)

        doc_encoder = Model(inputs=doc_input, outputs=doc_output)
        doc_encoder.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        return doc_encoder
    
    def preporcess(self, labels, char_dict=None, unknown_label='UNK'):
        if self.verbose > 3:
            print('-----> Stage: preprocess')
            
        self.build_char_dictionary(char_dict, unknown_label)
        self.convert_labels(labels)
    
    def process(self, df, x_col, y_col, 
                max_len_of_sentence=None, max_num_of_setnence=None, label2indexes=None, sample_size=None):
        if self.verbose > 3:
            print('-----> Stage: process')
            
        if sample_size is None:
            sample_size = 1000
        if label2indexes is None:
            if self.label2indexes is None:
                raise Exception('Does not initalize label2indexes. Please invoke preprocess step first')
            label2indexes = self.label2indexes
        if max_len_of_sentence is None:
            max_len_of_sentence = self.max_len_of_sentence
        if max_num_of_setnence is None:
            max_num_of_setnence = self.max_num_of_setnence

        x_preprocess, y_preprocess = self._transform_raw_data(
            df=df, x_col=x_col, y_col=y_col, label2indexes=label2indexes)
        
        x_preprocess, y_preprocess = self._transform_training_data(
            x_raw=x_preprocess, y_raw=y_preprocess,
            max_len_of_sentence=max_len_of_sentence, max_num_of_setnence=max_num_of_setnence)
        
        if self.verbose > 5:
            print('Shape: ', x_preprocess.shape, y_preprocess.shape)

        return x_preprocess, y_preprocess
    
    def build_model(self, char_dimension=16, display_summary=False, display_architecture=False, 
                    loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']):
        if self.verbose > 3:
            print('-----> Stage: build model')
            
        sent_encoder = self._build_sentence_block(
            char_dimension=char_dimension,
            max_len_of_sentence=self.max_len_of_sentence, max_num_of_setnence=self.max_num_of_setnence)
                
        doc_encoder = self._build_document_block(
            sent_encoder=sent_encoder, num_of_label=self.num_of_label,
            max_len_of_sentence=self.max_len_of_sentence, max_num_of_setnence=self.max_num_of_setnence, 
            loss=loss, optimizer=optimizer, metrics=metrics)
        
        if display_architecture:
            print('Sentence Architecture')
            IPython.display.display(SVG(model_to_dot(sent_encoder).create(prog='dot', format='svg')))
            print()
            print('Document Architecture')
            IPython.display.display(SVG(model_to_dot(doc_encoder).create(prog='dot', format='svg')))
        
        if display_summary:
            print(doc_encoder.summary())
            
        
        self.model = {
            'sent_encoder': sent_encoder,
            'doc_encoder': doc_encoder
        }
        
        return doc_encoder
    
    def train(self, x_train, y_train, x_test, y_test, batch_size=128, epochs=1, shuffle=True):
        if self.verbose > 3:
            print('-----> Stage: train model')
            
        self.get_model().fit(
            x_train, y_train, validation_data=(x_test, y_test), 
            batch_size=batch_size, epochs=epochs, shuffle=shuffle)
        
#         return self.model['doc_encoder']

    def predict(self, x, return_prob=False):
        if self.verbose > 3:
            print('-----> Stage: predict')
            
        if return_prob:
            return self.get_model().predict(x_test)
        
        return self.get_model().predict(x_test).argmax(axis=-1)
    
    def get_model(self):
        return self.model['doc_encoder']

In [9]:
"""
    Maximum number of characters per sentence is 256.
    Maximum number of sentence is 5
"""

char_cnn = CharCNN(max_len_of_sentence=256, max_num_of_setnence=5)

"""
    First of all, we need to prepare meta information including character dictionary 
    and converting label from text to numeric (as keras support numeric input only).
"""
char_cnn.preporcess(labels=df['category'].unique())

"""
    We have to transform raw input training data and testing to numpy format for keras input
"""
x_train, y_train = char_cnn.process(
    df=train_df, x_col='headline', y_col='category')
x_test, y_test = char_cnn.process(
    df=test_df, x_col='headline', y_col='category')

char_cnn.build_model()
char_cnn.train(x_train, y_train, x_test, y_test, batch_size=2048, epochs=10)

char_cnn.get_model().save('./char_cnn_model.h5')

-----> Stage: preprocess
Totoal number of chars: 83
First 3 char_indices sample: {'u': 1, ';': 2, 'h': 3}
First 3 indices_char sample: {0: 'UNK', 1: 'u', 2: ';'}
Label to Index:  {'m': 3, 'e': 2, 'b': 0, 't': 1}
Index to Label:  {0: 'b', 1: 't', 2: 'e', 3: 'm'}
-----> Stage: process
Number of news: 337935
Actual max sentence: 15
unknown_value: 0
Shape:  (337935, 5, 256) (337935,)
-----> Stage: process
Number of news: 84484
Actual max sentence: 12
unknown_value: 0
Shape:  (84484, 5, 256) (84484,)
-----> Stage: build model
-----> Stage: train model
Train on 337935 samples, validate on 84484 samples
Epoch 1/10
337935/337935 [==============================] - 11080s 33ms/step - loss: 1.2129 - acc: 0.4373 - val_loss: 1.1089 - val_acc: 0.4961
Epoch 2/10
337935/337935 [==============================] - 11040s 33ms/step - loss: 1.0237 - acc: 0.5594 - val_loss: 0.9715 - val_acc: 0.5940
Epoch 3/10
337935/337935 [==============================] - 11017s 33ms/step - loss: 0.7820 - acc: 0.6824 - va

In [10]:
char_cnn_model_loaded = load_model('./char_cnn_model.h5')
char_cnn_model_loaded.predict(x_test)

array([[  9.91751850e-01,   2.97714826e-02,   1.48440502e-03,
          6.78197597e-04],
       [  4.47248220e-01,   9.31121230e-01,   6.35791337e-03,
          2.18354538e-03],
       [  9.97535825e-01,   3.43346526e-03,   5.12215891e-04,
          1.37654960e-03],
       ..., 
       [  1.38067767e-01,   9.78998482e-01,   1.27230678e-02,
          3.33724148e-03],
       [  1.94361171e-04,   2.12071711e-04,   9.99878764e-01,
          9.05996567e-05],
       [  8.41215439e-03,   1.44472681e-02,   9.66172934e-01,
          1.35373905e-01]], dtype=float32)

# Conclusion
Character Embedding is a brilliant design for solving lots of text classification. It resolved some word embedding. FAIR did a further step. They introduced to use subword embedding to build fastText. 

This is some comment on Character Embedding as it does not include any word meaning but just using characters. We may include both Character Embedding and Word Embedding together to solve our NLP problem.